In [11]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


In [5]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
from nltk.stem.porter import PorterStemmer
stmer= PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stmer.stem(word.lower()) 

def bag_of_words(tokenized_sentence, all_words):
    tokenized_sentence = [stem(w) for w in tokenized_sentence]
    bag = np.zeros(len(all_words), dtype = np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0
            
    return bag

In [7]:
import json
with open('intents.json', 'r') as f:
    intents =json.load(f)

all_words = []
tags = []
xy = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w,tag))


ignore_words = ['?','!','.',',']
all_words = [stem(w) for w in all_words if w not in ignore_words]

all_words = sorted(set(all_words))
tags = sorted(set(tags))

xtrain = []
ytrain = []

for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    xtrain.append(bag)
    label = tags.index(tag)
    ytrain.append(label)

xtrain = np.array(xtrain)
ytrain = np.array(ytrain) 

class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(xtrain)
        self.x_data = xtrain
        self.y_data = ytrain
        
    def __getitem__(self,index):
        
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.n_samples
    
#Hyperparameters
batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(xtrain[0])
num_classes = len(xtrain[0]) 

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size = 8, shuffle = True, num_workers = 0)
print("hello")
    

hello


In [8]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out
        

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model  =  NeuralNet(input_size, hidden_size, output_size).to(device)

In [10]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=.001)
num_epochs =1000

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print (f'final loss, Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 1.5117
Epoch [200/1000], Loss: 0.1433
Epoch [300/1000], Loss: 0.0534
Epoch [400/1000], Loss: 0.0037
Epoch [500/1000], Loss: 0.0034
Epoch [600/1000], Loss: 0.0035
Epoch [700/1000], Loss: 0.0035
Epoch [800/1000], Loss: 0.0034
Epoch [900/1000], Loss: 0.0019
Epoch [1000/1000], Loss: 0.0012
final loss, Epoch [1000/1000], Loss: 0.0012


In [11]:
data = {
    "model_state": model.state_dict(),
    "input_size" : input_size,
    "output_size": output_size,
    "hidden_size": hidden_size,
    "all_words"  : all_words,
    "tags"       : tag
    
}

FILE = "data.pth"
torch.save(data, FILE)
model_state = data["model_state"]

In [12]:
model.load_state_dict(model_state)
model.eval()

NeuralNet(
  (l1): Linear(in_features=54, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=7, bias=True)
  (relu): ReLU()
)

In [13]:
bot_name = "Babu"
import random

print( "Let's chat! type 'quit' to exit")
while True:
    sentence = input("You: ")
    if sentence == 'quit':
        break
    
    sentence = tokenize(sentence)
    x= bag_of_words(sentence,all_words)
    x = x.reshape(1, x.shape[0])
    x = torch.from_numpy(x)
    
    output = model(x)
    _, predicted = torch.max(output,dim=1)
    tag = tags[predicted.item()]
    
    probs = torch.softmax(output, dim =1)
    prob = probs[0][predicted.item()]
    
    if prob.item()>.75: 
         for intent in intents["intents"]:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}") 
    
    
      


Let's chat! type 'quit' to exit


KeyboardInterrupt: Interrupted by user

In [14]:
def get_response(msg):
    
    
    
    msg = tokenize(msg)
    x= bag_of_words(msg,all_words)
    x = x.reshape(1, x.shape[0])
    x = torch.from_numpy(x)
    
    output = model(x)
    _, predicted = torch.max(output,dim=1)
    tag = tags[predicted.item()]
    
    probs = torch.softmax(output, dim =1)
    prob = probs[0][predicted.item()]
    
    if prob.item()>.75: 
         for intent in intents["intents"]:
            if tag == intent["tag"]:
                return random.choice(intent['responses'])
    
    return "I don't understand..."
    


In [46]:
from tkinter import *

BG_GRAY = "#ABB2B9"
BG_COLOR = "#17202A"
TEXT_COLOR = "#EAECEE"

FONT = "Helvetica 14"
FONT_BOLD =  "Helvetica 13 bold"

class ChatApplication:
    
    def __init__(self):
        self.window = Tk()
        self._setup_main_window()
        
    def run(self):
        self.window.mainloop() 
    
    
    def _setup_main_window(self):
        self.window.title("Chat")
        self.window.resizable(width =False, height = False)
        self.window.configure(width = 470, height = 550, bg= BG_COLOR)
        
        #head label
        
        head_label = Label(self.window, bg = BG_COLOR, fg = TEXT_COLOR,
                           text = "welcome" , font = FONT_BOLD, pady = 10) 
        
        head_label.place(relwidth=1)
        
        #tinny Divider
        line = Label(self.window, width = 450, bg = BG_GRAY)
        line.place (relwidth =1, rely = 0.07, relheight = 0.012) 
        
        #text widget
        
        self.text_widget = Text(self.window, width = 20, height =2, bg = BG_COLOR, fg = TEXT_COLOR,
                                font = FONT, padx = 5, pady = 5) 
        
        self.text_widget.place(relheight= .745, relwidth =1, rely = 0.08)
        self.text_widget.configure(cursor = "arrow",state = DISABLED)
        
        # scroll bar
        
        scrollbar = Scrollbar(self.text_widget)
        scrollbar.place(relheight =1, relx = .974)
        scrollbar.configure(command = self.text_widget.yview)
        
        #bottom label
        
        bottom_label = Label(self.window, bg = BG_GRAY, height = 80)
        
        bottom_label.place(relwidth = 1, rely = .825)
        
        #massage entry box
        
        self.msg_entry = Entry(bottom_label, bg= "#2C3E50", fg = TEXT_COLOR, font = FONT)
        
        self.msg_entry.place(relwidth= .74, relheight = .06, rely = .008, relx = .011) 
        self.msg_entry.focus()
        self.msg_entry.bind("<Return>",self._on_enter_pressed) 
        
        #send botton
        
        send_button = Button(bottom_label, text = "Send" , font = FONT_BOLD, width = 20, bg = BG_GRAY , 
                             command = lambda: self._on_enter_pressed(None))
        send_button.place(relx = 0.77, rely=.008, relheight =.06, relwidth = .22)
        
    def _on_enter_pressed(self,event):
        msg = self.msg_entry.get()
        self._insert_message(msg, "You")
    
    def _insert_message(self,msg, sender):
        
        if not msg:
            return
        
        self.msg_entry.delete(0, END)
        msg1 = f"{sender}: {msg}\n\n"
        self.text_widget.configure(state = NORMAL)
        self.text_widget.insert( END, msg1)
        self.text_widget.configure(state = DISABLED) 
        
        msg2 = f"{bot_name}: {get_response(msg)}\n\n"
        self.text_widget.configure(state = NORMAL)
        self.text_widget.insert( END, msg2)
        self.text_widget.configure(state = DISABLED) 
        
        self.text_widget.see(END)
        
        
        


In [47]:

if __name__ == "__main__":
    app = ChatApplication()
    app.run()